# Topic Modeling, An Outline of a Workflow

This is aimed at giving students a clear example of a topic-modeling workflow.  Please feel free to follow this outline.


As a reminder:

1. When we are doing _any_ kind of ML, we want to try the simplest thing possible to establish a baseline.
1. By knowing a few useful features of sklearn and a few good snippets of nltk, we can accomplish a lot with very little code.
1. The steps of all topic modeling will always be:
    1. Gather your data (scrape it, process the Kaggle data set, etc) and store it into a convenient format, like a csv.
    1. Read in that csv
    1. Preprocess it (this data set is already slightly preprocessed, which is annoying, but okay)
    1. Vectorize documents into document-word matrix or DataFrame
    1. Feed that into our topic modeler.
    1. Print the topics, so we can see if they are interpretable.
    1. Repeat this process in an effort to create better and more interpretable topics.

In [1]:
import pandas as pd
import numpy as np

# Part 1: Wrangling the Text Data Together

I found a random dataset on Kaggle, it's a data set on movie reviews from the 1990's and early 2000's.  I put all of it into a csv, which you can load below. (Each document is an individual user review of a film.)

In [2]:
reviews_df = pd.read_csv("reviews.csv")

reviews_df.head()

docs sentiment
0  films adapted from comic books have had plenty...       pos
1  every now and then a movie comes along from a ...       pos
2  you've got mail works alot better than it dese...       pos
3  " jaws " is a rare film that grabs your attent...       pos
4  moviemaking is a lot like being the general ma...       pos

# Part 2: Topic Modeling Utility

Remember when we talked about pipelines?  Yeah, I'm just going to write one so I don't need to copy/paste large amounts of code all over the place.

All this does is:

1. Takes in docs, preprocessor, vectorizer, and topic modeler --each of your choosing (though gensim LDA will not work).
1. Applies these in order.
1. Prints out the top 15 words for each topic that was found from training.

__This is all we will ever need to check to see if our topic modeling approach is working.__  I'll repeat this now and then again throughout this notebook:  Your end goal is to have interpretable topics.  So let's just write a function that automates that process of making and displaying topics!  Now we just iterate by playing around with these parameters.

In [3]:
def make_topics(docs, preprocessor, vectorizer, topic_modeler, print_n_words=15):
    """A very simple pipeline."""
    
    # Apply preprocessor, vectorizer, and topic modeler.
    if preprocessor is not None:
        docs = docs.apply(preprocessor)
    
    # Vectorize documents into a document-word matrix.
    doc_word_vectors = vectorizer.fit_transform(docs)
    
    # Fit the topic model.
    doc_topic_vectors = topic_modeler.fit_transform(doc_word_vectors)
    
    # Print the topics.
    vocab = vectorizer.get_feature_names()
    for idx, topic in enumerate(topic_modeler.components_):
        # Select the top 15 words in vocab for this topic.
        top_words = [vocab[i].upper() for i in topic.argsort()[:-print_n_words-1:-1]]
        print(f"Topic {idx}:\n", ", ".join(top_words), "\n")
    
    return doc_topic_vectors

# Part 3: Make a Baseline Topic Model

What's our next step?  Try out fancy preprocessing?  Try out LDA topic modeling?  Start playing around with min_df, max_df?  Try out stemming/lemmatization?  Try out some bigrams?  Try looking for stuff to remove with regex?  Play around with the stop words?  Look for commonly misspelled words?  Do NER to find proper nouns?

_Nope, no, no, no, no, and also no._  __The first thing you must do is make baseline topics__.  Just like with supervised learning, you cannot know if any choice you are making is _improving anything unless you can compare it to something else_.  So let's make a fast, effortless baseline model, and then we can see what's going wrong (or if we're lucky, right) and then we have something to compare our fancy techniques against.  This way, we can make sure that we're working in a positive direction, and also know when we're hitting dead ends with a technique we're trying.

So let's setup that baseline model.  I've already spoken in class that a really good prior to have here is:  Tfidf vectorizer + NMF topic modeler.  Let's do that.

In [4]:
# Let's import them:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [5]:
docs = reviews_df["docs"]
preprocessor = None
vectorizer = TfidfVectorizer()
topic_modeler = NMF(10, random_state=10, max_iter=1000)  # NOTE: I use a random state here to make the results deterministic.

make_topics(docs, preprocessor, vectorizer, topic_modeler);

Topic 0:
 THE, OF, IN, AND, TO, IS, ON, AS, BY, FROM, AN, FILM, THAT, ARE, MOST 

Topic 1:
 HE, HIS, TO, AND, HIM, IS, IN, HAS, WHO, THE, WITH, AS, LIFE, MAN, FOR 

Topic 2:
 IS, OF, IT, FILM, AND, THE, THIS, THAT, NOT, IN, ITS, TO, BE, FILMS, ONE 

Topic 3:
 HER, SHE, AND, IN, THE, TO, HUSBAND, IS, WITH, OF, WHO, WOMAN, LIFE, FOR, MOTHER 

Topic 4:
 WAS, IT, THE, MOVIE, THAT, TO, MY, ME, THIS, HAD, BAD, SO, BUT, NOT, WHEN 

Topic 5:
 THEY, AND, THEIR, ARE, TO, THE, THEM, TWO, HAVE, DO, IN, ABOUT, FOR, EACH, OF 

Topic 6:
 DISNEY, THE, MULAN, ANIMATED, TARZAN, AND, OF, ANIMATION, FEATURE, TO, FAMILY, WITH, BY, ANASTASIA, WAS 

Topic 7:
 YOU, YOUR, IF, THE, TO, THAT, KNOW, IT, SEE, RE, MOVIE, WHAT, WILL, HAVE, THIS 

Topic 8:
 JACKIE, CHAN, ACTION, KONG, HONG, DAMME, THE, MARTIAL, VAN, OF, ARTS, AND, IN, HAS, RUSH 

Topic 9:
 SCREAM, HORROR, THE, AND, SLASHER, OF, SUMMER, GENRE, DID, LAST, JULIE, WAS, SEQUEL, KNOW, FILM 



### NOTE:

Wow!  These are some truly awful, garbage topics!  It can really only be up hill from here....

---
## Topic-Modeling Experiment #1: One Small Change...


What do we notice about the above topics?

We should obviously let the data guide us, and __I'd say it's telling us we need to use a robust list of English stop words__, because we can see that our topics are dominated by non-descript words that frequently appear in written English.  We may also have too many topics (since there's a "he" topic and a "she" topic, that seems like it's really splitting hairs).

__But let's do the the simplest thing first, because that looks like it will have a big impact__.  Always think this way.  Look at what's going wrong with your topics.  Fix the biggest problem first, only after that should you try other refinements.  (Avoid doing multiple changes at a time, because you don't know which one had the largest important effect on the topics.)

In [6]:
docs = reviews_df["docs"]
preprocessor = None
vectorizer = TfidfVectorizer(stop_words="english")  # Single change to add in common English stop words.
topic_modeler = NMF(10, random_state=10)

make_topics(docs, preprocessor, vectorizer, topic_modeler);

Topic 0:
 MOVIE, BAD, GOOD, MOVIES, JUST, REALLY, LIKE, KNOW, VE, PLOT, THING, SEEN, DON, THINK, GOT 

Topic 1:
 LIFE, MAN, NEW, WIFE, LOVE, BOY, FAMILY, YOUNG, FATHER, STORY, GIRL, MOTHER, JOHN, TAKES, OLD 

Topic 2:
 FILM, FILMS, STORY, DIRECTOR, CHARACTERS, LIKE, TIME, PICTURE, SEEN, BEST, GREAT, JUST, GOING, PLOT, REALLY 

Topic 3:
 JACKIE, CHAN, ACTION, DAMME, KONG, HONG, MARTIAL, VAN, ARTS, RUSH, LI, FILMS, SCENES, HOUR, TARANTINO 

Topic 4:
 SCREAM, HORROR, SLASHER, SUMMER, DID, KNOW, JULIE, GENRE, SEQUEL, WILLIAMSON, KILLER, TEEN, URBAN, MOVIES, CRAVEN 

Topic 5:
 ALIEN, ALIENS, MARS, EARTH, SPACE, PLANET, SCIENCE, FI, SCI, FICTION, SPECIES, ACTION, EVENT, RIPLEY, HORIZON 

Topic 6:
 STAR, WARS, PHANTOM, MENACE, TREK, LUCAS, JEDI, EFFECTS, SPECIAL, EPISODE, GEORGE, TRILOGY, SERIES, FILM, KU 

Topic 7:
 DISNEY, ANIMATED, MULAN, TARZAN, ANIMATION, FEATURE, FAMILY, STORY, KING, ANASTASIA, POCAHONTAS, LOST, MERMAID, CARTOON, VOICED 

Topic 8:
 TRUMAN, CARREY, BURBANK, JIM, WORLD, L

### Wow, this is way better, no?



---
# Experiment #2: Change Number of Topics

Okay, so stop words are a must for this data.  These look like okay topics.  When you have meaningful topics, now we should see if we can get a little improvement by tweaking the number of topics.  Let's see what happens if we make 5 and 15 topics.

In [7]:
# Five topics.

docs = reviews_df["docs"]
preprocessor = None
vectorizer = TfidfVectorizer(stop_words="english")
topic_modeler = NMF(5, random_state=10)  # NOTE: I use a random state here to make the results deterministic.

make_topics(docs, preprocessor, vectorizer, topic_modeler);

Topic 0:
 MOVIE, BAD, JUST, GOOD, LIKE, REALLY, MOVIES, VE, KNOW, PLOT, THING, SEEN, THINK, DON, DIDN 

Topic 1:
 FILM, STORY, LIFE, LOVE, MAN, NEW, TIME, DIRECTOR, FAMILY, CHARACTERS, LIKE, WORLD, YOUNG, WIFE, CHARACTER 

Topic 2:
 STAR, ALIEN, FILM, WARS, ALIENS, FICTION, TREK, SCIENCE, EFFECTS, FILMS, PLANET, SCI, FI, SPACE, EARTH 

Topic 3:
 JACKIE, CHAN, ACTION, DAMME, KONG, HONG, VAN, MARTIAL, ARTS, FILMS, RUSH, LI, SCENES, TARANTINO, COP 

Topic 4:
 SCREAM, HORROR, SLASHER, SUMMER, FILM, DID, KNOW, GENRE, JULIE, SEQUEL, KILLER, WILLIAMSON, TEEN, FILMS, URBAN 



Not terrible, though it does look a little compressed.  What if we try out the other direction and look at 15 topics?

In [8]:
# Fifteen topics.

docs = reviews_df["docs"]
preprocessor = None
vectorizer = TfidfVectorizer(stop_words="english")
topic_modeler = NMF(15, random_state=10, max_iter=1000)  # NOTE: I use a random state here to make the results deterministic.

make_topics(docs, preprocessor, vectorizer, topic_modeler);

Topic 0:
 MOVIE, BAD, JUST, GOOD, REALLY, MOVIES, LIKE, KNOW, THING, VE, PLOT, THINK, SEEN, DON, DIDN 

Topic 1:
 LIFE, LOVE, COMEDY, FAMILY, BOY, FATHER, ROMANTIC, MOTHER, SCHOOL, TOWN, GIRL, OLD, BEAUTIFUL, HIGH, YEAR 

Topic 2:
 FILM, FILMS, STORY, DIRECTOR, CHARACTERS, LIKE, TIME, SEEN, PICTURE, GREAT, BEST, JUST, GOING, WATCHING, DISASTER 

Topic 3:
 JACKIE, CHAN, MARTIAL, ARTS, KONG, HONG, ACTION, TARANTINO, RUSH, MR, HOUR, BROWN, MOVIES, FILMS, SCENES 

Topic 4:
 SCREAM, HORROR, SLASHER, SUMMER, DID, JULIE, KNOW, GENRE, SEQUEL, WILLIAMSON, TEEN, KILLER, URBAN, CRAVEN, COLLEGE 

Topic 5:
 VAMPIRES, VAMPIRE, CARPENTER, JOHN, BLADE, HORROR, BOOK, COMIC, JACK, MOVIES, CROW, WOODS, MARS, GHOSTS, HALLOWEEN 

Topic 6:
 WARS, STAR, PHANTOM, MENACE, JEDI, LUCAS, EFFECTS, SPECIAL, EPISODE, GEORGE, TRILOGY, NEESON, DARTH, LIAM, MCGREGOR 

Topic 7:
 DISNEY, MULAN, ANIMATED, TARZAN, ANIMATION, FEATURE, KING, ANASTASIA, STORY, FAMILY, POCAHONTAS, MERMAID, LOST, CARTOON, VOICED 

Topic 8:
 TRU

Well, what do you think?  Personally, I'd say this is a great example of tweaking topics.  Okay, so first note that a lot of the topics are less "coherent", I'd say, but notice that we now have a war movies topic toward the bottom.  This happens often where you get one kind of nice new topic, but the other topics are starting to decay and become more about particular movies than genres or common discussion points of movies.

__Ultimately, you're the judge__.  You get to make the final call, but for me personally, I'd say 15 is too many topics, and the topics are beginning to degrade here.

I'm going to play around with different numbers of topics, and see what I "feel" are good, clean topics.

---
# My Topics

Okay, I did those experiments, and for me, I like __9__ topics.  Basically everything between 8 and 13 seemed okay, but for me 9 seemed to have a good balance of everything.

In [9]:
docs = reviews_df["docs"]
preprocessor = None
vectorizer = TfidfVectorizer(stop_words="english")
topic_modeler = NMF(9, random_state=10, max_iter=2000)

make_topics(docs, preprocessor, vectorizer, topic_modeler);

Topic 0:
 MOVIE, BAD, GOOD, JUST, MOVIES, REALLY, LIKE, KNOW, VE, THING, PLOT, THINK, FUNNY, SEEN, DON 

Topic 1:
 LIFE, MAN, NEW, LOVE, COMEDY, WIFE, GETS, GIRL, BOY, FAMILY, FATHER, OLD, TOWN, JOHN, PLAYS 

Topic 2:
 FILM, FILMS, STORY, CHARACTERS, DIRECTOR, LIKE, TIME, PICTURE, SEEN, BEST, GREAT, JUST, PLOT, WAR, GOING 

Topic 3:
 JACKIE, CHAN, ACTION, DAMME, KONG, HONG, MARTIAL, VAN, ARTS, RUSH, LI, FILMS, SCENES, COP, HOUR 

Topic 4:
 SCREAM, HORROR, SLASHER, SUMMER, DID, JULIE, KNOW, GENRE, SEQUEL, WILLIAMSON, TEEN, KILLER, URBAN, MOVIES, CRAVEN 

Topic 5:
 TRUMAN, CARREY, BURBANK, JIM, WORLD, LIFE, ACE, PERFECT, VENTURA, SEAHAVEN, WEIR, REALITY, PLEASANTVILLE, PEOPLE, HOLLYWOOD 

Topic 6:
 STAR, WARS, PHANTOM, MENACE, TREK, LUCAS, JEDI, EFFECTS, SPECIAL, EPISODE, GEORGE, TRILOGY, SERIES, FILM, BA 

Topic 7:
 DISNEY, ANIMATED, MULAN, TARZAN, ANIMATION, FEATURE, FAMILY, STORY, KING, ANASTASIA, POCAHONTAS, LOST, MERMAID, CARTOON, VOICED 

Topic 8:
 ALIEN, ALIENS, MARS, EARTH, SPACE

### NOTE:
For one above, I needed to increase the `max_iter` parameter because sklearn warned me the algorithm hadn't converged yet.  Don't worry if you need to do this, it just means the algorithm is requesting to be run a little bit longer, so let it.


---
# Final Interpretation

Here's what I would say my final topics are:


### General Movie Sentiment Topic:
> MOVIE, BAD, GOOD, JUST, MOVIES, REALLY, LIKE, KNOW, VE, THING, PLOT, THINK, FUNNY, SEEN, DON 

### Rom-Com Topic:
> LIFE, MAN, NEW, LOVE, COMEDY, WIFE, GETS, GIRL, BOY, FAMILY, FATHER, OLD, TOWN, JOHN, PLAYS 

### Misc Film Topic:
> FILM, FILMS, STORY, CHARACTERS, DIRECTOR, LIKE, TIME, PICTURE, SEEN, BEST, GREAT, JUST, PLOT, WAR, GOING 

### Martial Arts Topic:
> JACKIE, CHAN, ACTION, DAMME, KONG, HONG, MARTIAL, VAN, ARTS, RUSH, LI, FILMS, SCENES, COP, HOUR 

### Horror Movie Topic:
> SCREAM, HORROR, SLASHER, SUMMER, DID, JULIE, KNOW, GENRE, SEQUEL, WILLIAMSON, TEEN, KILLER, URBAN, MOVIES, CRAVEN 

### Jim Carrey Comedies (note: Jim Carrey was a comedian who was super famous in the 1990's, which is when many of these reviews were written):
> TRUMAN, CARREY, BURBANK, JIM, WORLD, LIFE, ACE, PERFECT, VENTURA, SEAHAVEN, WEIR, REALITY, PLEASANTVILLE, PEOPLE, HOLLYWOOD 

### STAR WARS topic:
> STAR, WARS, PHANTOM, MENACE, TREK, LUCAS, JEDI, EFFECTS, SPECIAL, EPISODE, GEORGE, TRILOGY, SERIES, FILM, BA 

### Animated Films Topic:
> DISNEY, ANIMATED, MULAN, TARZAN, ANIMATION, FEATURE, FAMILY, STORY, KING, ANASTASIA, POCAHONTAS, LOST, MERMAID, CARTOON, VOICED 

### Sci-Fi Film Topic:
> ALIEN, ALIENS, MARS, EARTH, SPACE, PLANET, SCIENCE, SCI, FI, FICTION, ACTION, SPECIES, EVENT, SHIP, HORIZON


---
# What Next?

First, we had a two "catch-all" topics that were just generally about films. This is completely okay and normal. You probably will never be able to get rid of catch-all topics; you just don't want most of your topics to be catch-all. If they start appearing too much, considering adding some of those words as stop words, or start playing around with the max_df parameter in TfidfVectorizer to get rid of words that appear "too much" in your corpus.


Second, we could continue to tweak this, but to be honest I'm not sure our results will improve drastically.  Here's what I would do if I really wanted to go the extra mile:

Possible recommendations:
1. Play around with `max_df` parameter of `TfidfVectorizer` to see if we can remove some of the "catch-all" topics.
1. We could throw "film", "movie", "films", etc, and other common film words into our stop words.  That might allow some new signal in.
1. Set `min_df` a little bit on the high side and try out some `ngram_range=(1,2)`, both are in `TfidfVectorizer`.  STARS_WARS is probably a good token, and maybe some others are, too. We want to try setting `min_df` a little bit high (maybe .01 or .02 to start with) so we don't generate too many bigrams.


Anti-recommendations:
1. There's almost no verbs or pluralizations here, so it probably would be a waste of time stem or lemmatize this corpus.  Maybe if we lemmatized we would see more signal from some verbs, but it probably won't.

As ever, let the data guide what preprocessing, fancy vectorization, or topic modeling techniques you use.  Never do things without the data pointing you in that direction, first!  This is a great way to mimimize the time you spend doing topic modeling.